In [4]:
%reset_selective -f b

# Este comando le indica a Python que las figuras se deben generar dentro de la misma Notebook,
##no en una ventana
## Solo para notebooks.
%matplotlib inline

from matplotlib import pyplot as plt
from matplotlib import colors
#from sklearn.svm import SVC
import numpy as np
import time
import glob
import pyart
import scipy.io as sio
import re
import funciones as fc
import pickle

#import tensorflow as tf
from sklearn.svm import SVC
def save(object,filename):
    with open(filename,"wb") as f:
        pickle.dump( object, f)
        
from collections import Counter

def vecinos(feature):
    f = np.c_[np.zeros(240), feature, np.zeros(240)].transpose()
    f = np.c_[np.zeros(242), f ,np.zeros(242)].transpose()
    x3=f[0:240].transpose()[0:240].transpose().ravel()
    x4=f[0:240].transpose()[1:241].transpose().ravel()
    x5=f[0:240].transpose()[2:242].transpose().ravel()
    x6=f[1:241].transpose()[0:240].transpose().ravel()
    x7=f[1:241].transpose()[2:242].transpose().ravel()
    x8=f[2:242].transpose()[0:240].transpose().ravel()
    x9=f[2:242].transpose()[1:241].transpose().ravel()
    x10=f[2:242].transpose()[2:242].transpose().ravel()
    return(x3,x4,x5,x6,x7,x8,x9,x10)


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [7]:
file = open("etiquetas.pickle",'rb')
etiquetas = pickle.load(file)
file = open("rayos.pickle",'rb')
rayos = pickle.load(file)
file = open("rayos_post.pickle",'rb')
rayos_post = pickle.load(file)

In [20]:
X_n=[]
y_n=[]
for j in range(345):
    #print j
    
    y=etiquetas[j].data.ravel()
    valor=np.unique(y)
    #print(valor.size)
    if (valor.size > 1):
        a=rayos[j].ravel()
        b=rayos_post[j].ravel()
        v=vecinos(rayos[j])
        w=vecinos(rayos_post[j])

        feat=20
        samp=a.size
        X=np.zeros((feat,samp))

        X[0]=a
        X[1]=b

        for i in range(0,7):
            X[i+2]=v[i]
            X[i+10]=w[i]


        X=X.transpose()
        clf = SVC(C=5.0)

        clf.fit(X, y)

        for k in (clf.support_):
            X_n.append(X[k])
            y_n.append(y[k])

In [21]:
print(len(y_n),etiquetas[0].data.size)

(78453, 57600)


In [ ]:
X2=np.zeros((len(y_n),20))
y2=y_n
for i in range (len(y_n)):
    X2[i]=X_n[i]


In [ ]:
clf = SVC(C=5.0)

clf.fit(X2, y2)

In [ ]:
for i in (1,):
    X=np.zeros((feat,240*240))
    a=rayos[5].ravel()
    b=rayos_post[5].ravel()
    X[0]=a
    X[1]=b
    X[2]=v[0]
    X[3]=v[1]
    X[4]=v[2]
    X[5]=v[3]
    X[6]=v[4]
    X[7]=v[5]
    X[8]=v[6]
    X[9]=v[7]

    X[10]=w[0]
    X[11]=w[1]
    X[12]=w[2]
    X[13]=w[3]
    X[14]=w[4]
    X[15]=w[5]
    X[16]=w[6]
    X[17]=w[7]

    X = X.transpose()
    resultado = clf.predict(X)
    test=etiquetas[i].data.ravel()-resultado


    plt.figure(1)

    plt.figure(figsize=(10,10))

    plt.subplot(221)
    plt.title('Prediccion')
    plt.imshow(resultado.reshape(240,240),origin='lower',cmap='Paired')

    plt.subplot(222)
    plt.title('Real')
    plt.imshow(etiquetas[i].data,origin='lower',cmap='Paired')


    plt.show()
    